## Plot a particular set of correlations extracted from the code.
## Compare to Poisson simulations of area distortions.
## Craig Lage - 16Sep19

In [ ]:
from pylab import *
import os, sys, glob, time, scipy
from scipy import stats
import eups
from lsst.daf.persistence import Butler
%matplotlib inline

In [ ]:
REPO_DIR = '/home/cslage/Research/LSST/code/notebooks/notebooks_2019_02_06/stand_alone_13feb19/flats_repo_itl_quad_zero_large_bias'
bfk_butler = Butler(REPO_DIR+'/rerun/test')
# Note ITL uses raftName R02 and detector 2
# E2V uses raftName R00 and detector 0
bf_kernel = bfk_butler.get('brighterFatterKernel', dataId={'raftName': 'R02', 'detectorName': 'S00',
                                                              'detector': 2})
means = bf_kernel.means
xcorrs = bf_kernel.xcorrs
meanXcorrs = bf_kernel.meanXcorrs

In [ ]:
# Look at the values for a particular flux
amp = 'C14'
flux = 18

print(" %s mean = "%(amp),means[amp][flux])
print(" %s xcorr[0,0] = "%(amp),xcorrs[amp][flux][0,0])
print(" %s xcorr[1,0] = "%(amp),xcorrs[amp][flux][1,0])
print(" %s xcorr[0,1] = "%(amp),xcorrs[amp][flux][0,1])
print(" %s meanXcorr[0,0] = "%(amp),meanXcorrs[amp][8,8]) 
print(" %s meanXcorr[0,1] = "%(amp),meanXcorrs[amp][8,9])

In [ ]:
# Read the simulated areas from the Poisson simulator

def ReadConfigFile(filename):
    # This reads the Poisson simulator config file for
    # the settings that were run
    # and returns a dictionary with the values

    with open(filename,'r') as file:
        lines=file.readlines()
    lines = [ l.strip() for l in lines ]
    lines = [ l.split() for l in lines if len(l) > 0 and l[0] != '#' ]
    for line in lines:
        if line[1] != '=':
            print("Check line: ",line)
            raise IOError("Error reading config file %s"%filename)
    config = {}
    for line in lines:
        try:
            # First get the ordered pairs
            config.update({line[0]:[eval(line[2]), eval(line[3])]})
        except:
            try:
                # Then get the numeric values
                config.update({line[0]:eval(line[2])})
            except:
                try:
                    # Last, get the strings
                    config.update({line[0]:str(line[2])})
                except:
                    pass
    return config


def ReadAreaFile(filename, nx, ny, nxcenter, nycenter, Area_0):
    # This reads the correlation data file
    # and returns an array with the expected correlations
    area = zeros([nx, ny])
    file = open(filename, 'r')
    lines = file.readlines()
    file.close()
    lines.remove(lines[0]) # Strip the title line    
    for line in lines:
        items = line.split()
        i = int(items[0])
        j = int(items[1])
        area[i,j] = float(items[2])

    sim = zeros([8,8])
    num = zeros([8,8], dtype = int)    
    try:
        for i in range(nx):
            for j in range(ny):
                ii = abs(i - nxcenter)
                jj = abs(j - nycenter)
                sim[jj,ii] += (area[i,j] - Area_0) / Area_0
                num[jj,ii] += 1
        for i in range(8):
            for j in range(8):
                if num[i,j] >0:
                    sim[i,j] /= float(num[i,j])
    except:
        pass
    return [area,sim]

# Read the .cfg file
home_dir = '/home/cslage/Research/LSST/code/poisson/Poisson_CCD_Hole20_Test11E_ITL_Corr/'
run = 0
configfile = home_dir + 'data/run100K_Vp38/pixel.cfg'
ConfigData = ReadConfigFile(configfile)
outputfilebase = ConfigData["outputfilebase"]
outputfiledir = ConfigData["outputfiledir"]
Nx = ConfigData["PixelBoundaryNx"]
Ny = ConfigData["PixelBoundaryNy"]
XCenter = ConfigData["FilledPixelCoords_0_0"][0]
YCenter = ConfigData["FilledPixelCoords_0_0"][1]
PixelSizeX = ConfigData["PixelSizeX"]
PixelSizeY = ConfigData["PixelSizeY"]
NxCenter = int((XCenter - ConfigData["PixelBoundaryLowerLeft"][0]) / PixelSizeX)
NyCenter = int((YCenter - ConfigData["PixelBoundaryLowerLeft"][1]) / PixelSizeY)
Area_0 = 99.9974
NumAngles = 4 * ConfigData["NumVertices"] + 4
NumElec = ConfigData["CollectedCharge_0_0"]

filename = home_dir + outputfiledir + '/' + outputfilebase +'_%d_Area'%run + '.dat'
[area,sim] = ReadAreaFile(filename, Nx, Ny, NxCenter, NyCenter, Area_0)

sim /= float(NumElec)

In [ ]:
print(sim.shape)
print(meanXcorrs[amp].shape)

In [ ]:
# Plot the correlations as a function of pixel 
NumPairs = 5
NumFluxes = 20
amp = 'C14'
posrs = []
negrs = []
poscs = []
negcs = []
fitrs = []
fitcs = []
poserrs = []
negerrs = []
FOM = 0.0
Num_FOM = 0.0
for ii in range(17):
    for jj in range(17):
        r2 = (ii-8)**2 + (jj-8)**2
        value = meanXcorrs[amp][ii,jj]
        n = NumFluxes - 1
        cov = []
        
        
        for m in range(NumPairs):
            i = n * NumPairs + m
            xcorr = xcorrs[amp][i][abs(ii-8),abs(jj-8)]
            cov.append(xcorr)
        cov = array(cov)
        if ii == 8 and jj == 8:
            negcs.append(abs(value))
            negrs.append(0.85)
            negerrs.append(abs(cov.std()))
        elif value < 0.0:
            poscs.append(abs(value))
            posrs.append(r2)
            poserrs.append(abs(cov.std()))
            if r2 > 1.1 and r2 < 20.0:
                fitrs.append(log10(r2))
                fitcs.append(log10(abs(value)))
        else:
            negcs.append(abs(value))
            negrs.append(r2)
            negerrs.append(abs(cov.std()))
        if (ii-8>=0) and (ii-8<8) and (jj-8>=0) and (jj-8 <8):
            FOM_contrib = ((value + sim[ii-8,jj-8]) / cov.std())**2
            print("i = %d, j = %d, data = %.6g sim = %.6g, FOM_contrib = %.6g"%(ii-8,jj-8,-value,sim[ii-8,jj-8],FOM_contrib))
            FOM += FOM_contrib
            Num_FOM += 1.0
            
            
chisquared = FOM / Num_FOM
print("Chi-squared = %.2f"%chisquared  )           
slope, intercept, r_value, p_value, std_err = stats.linregress(fitrs,fitcs)
#slope = -1.35
print(slope,intercept, r_value)

possimrs = []
negsimrs = []
possimcs = []
negsimcs = []
for i in range(8):
    for j in range(8):
        r2 = i**2 + j**2
        value = sim[i,j]
        if i == 0 and j == 0:
            negsimcs.append(abs(value))
            negsimrs.append(0.85)
        elif value > 0.0:
            possimcs.append(abs(value))
            possimrs.append(r2)
        else:
            negsimcs.append(abs(value))
            negsimrs.append(r2)

            
            
            
figure(figsize=(16,8))
title("Covariance Matrix", fontsize = 24)
errorbar(posrs, poscs, yerr=poserrs, color='blue', marker = 'o', markersize = 8, ls = '', label = 'Positive Meas')
errorbar(negrs, negcs, yerr=negerrs, color='red', marker = 'o', markersize = 8, ls = '', label = 'Negative Meas')

scatter(possimrs, possimcs, color='green', marker = 'x', s=200, label = 'Positive Sims')
scatter(negsimrs, negsimcs, color='magenta', marker = 'x', s=200, label = 'Negative Sims')

text(1.2, 5E-6, "C00: Meas = %.4g, Sim = %.4g"%(-meanXcorrs[amp][8,8],sim[0,0]), fontsize = 24)
text(1.2, 2.5E-6, "C01: Meas = %.4g, Sim = %.4g"%(-meanXcorrs[amp][9,8],sim[1,0]), fontsize = 24)
text(1.2, 1.25E-6, "C10: Meas = %.4g, Sim = %.4g"%(-meanXcorrs[amp][8,9],sim[0,1]), fontsize = 24)
text(1.2, 6.25E-7, "C11: Meas = %.4g, Sim = %.4g"%(-meanXcorrs[amp][9,9],sim[1,1]), fontsize = 24)
text(1.2, 3.10E-7, "$\chi^2$/DOF = %.2f"%chisquared, fontsize = 24)
xplot = linspace(1.0,200,100)
yplot = 10.0**intercept * xplot**(slope)
#plot(xplot,yplot,ls='--',color='green')
xscale('log')
yscale('log')
xlim(0.8,100)
ylim(1E-10,1E-5)
xticks([1.0, 10.0, 100.0], fontsize=24)
yticks([1E-10, 1E-9, 1E-8, 1E-7, 1E-6, 1E-5], fontsize=24)
xlabel("$i^2 + j^2$", fontsize = 24)
ylabel("Covariance or $\delta$ Area/Area", fontsize = 24)

def ModelCov(ii,jj,intercept,slope):
    r2 = float(ii**2 + jj**2)
    return 10.0**intercept * r2**(slope)
covsteps = 8
modelrs = []
modelcs = []
for ii in range(covsteps):
    for jj in range(covsteps):
        r2 = ii**2 + jj**2
        if ii > 2 or jj > 2:
            cov_mean = ModelCov(ii,jj,intercept,slope)
            modelrs.append(r2)
            modelcs.append(cov_mean)
#scatter(modelrs,modelcs,marker='x',s=100,label='Model')
legend(fontsize=24)#bbox_to_anchor=(0.80, 0.80), loc=2, borderaxespad=0., fontsize = 12)

savefig(REPO_DIR+'/rerun/test/plots/Correlations_Sims_21Oct19_%s.pdf'%amp)



#### 